In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import accuracy_score, log_loss, f1_score, classification_report
from IPython.display import display

import pickle
import gc
import os
import datetime as dt
from datetime import datetime
import json 


In [2]:
# 最大列数を None（制限なし）に
pd.set_option('display.max_columns', None)

# 最大行数も None に
pd.set_option('display.max_rows', None)


In [ ]:
import os

# Notebook用にスクリプトの場所を手動で設定
BASE_DIR = r"C:\Users\image\Portfolio\Soccer analysis\soccer_pipeline\notebooks"

# プロジェクトのルート（soccer_pipeline）
ROOT_DIR = os.path.abspath(os.path.join(BASE_DIR, ".."))

data_file = os.path.join(ROOT_DIR,"data","processed_data.csv")

# データ読み込み

In [3]:
#データの読み込み
df = pd.read_csv(data_file)

df.head()

,fixture_id,date,season,home_team,away_team,home_score,away_score,status,home_shots_on_goal,home_possession,home_passes,home_passes_accuracy,home_fouls,home_corners,home_yellow_cards,home_red_cards,away_shots_on_goal,away_possession,away_passes,away_passes_accuracy,away_fouls,away_corners,away_yellow_cards,away_red_cards,home_last_position,home_last_won,home_last_drawn,home_last_lost,home_last_gf,home_last_ga,home_last_gd,home_last_points,away_last_position,away_last_won,away_last_drawn,away_last_lost,away_last_gf,away_last_ga,away_last_gd,away_last_points
0,710556,2021-08-13 19:00:00+00:00,2021,Brentford,Arsenal,2.0,0.0,FT,3.0,35.0,309.0,201.0,12.0,2.0,0.0,0.0,4.0,65.0,568.0,488.0,8.0,5.0,0.0,0.0,17,10,9,19,33,55,-22,39,8,18,7,13,55,39,16,61
1,710557,2021-08-14 14:00:00+00:00,2021,Burnley,Brighton,1.0,2.0,FT,3.0,36.0,259.0,181.0,10.0,7.0,2.0,0.0,8.0,64.0,518.0,424.0,7.0,6.0,1.0,0.0,17,10,9,19,33,55,-22,39,16,9,14,15,40,46,-6,41
2,710558,2021-08-14 14:00:00+00:00,2021,Chelsea,Crystal Palace,3.0,0.0,FT,6.0,62.0,678.0,623.0,15.0,5.0,0.0,0.0,1.0,38.0,423.0,363.0,11.0,2.0,0.0,0.0,4,19,10,9,58,36,22,67,14,12,8,18,41,66,-25,44
3,710559,2021-08-14 14:00:00+00:00,2021,Everton,Southampton,3.0,1.0,FT,6.0,48.0,337.0,235.0,13.0,6.0,2.0,0.0,3.0,52.0,370.0,256.0,15.0,8.0,0.0,0.0,10,17,8,13,47,48,-1,59,15,12,7,19,47,68,-21,43
4,710560,2021-08-14 14:00:00+00:00,2021,Leicester,Wolves,1.0,0.0,FT,5.0,56.0,584.0,505.0,6.0,5.0,1.0,0.0,3.0,44.0,443.0,366.0,10.0,4.0,2.0,0.0,5,20,6,12,68,50,18,66,13,12,9,17,36,52,-16,45


In [4]:
df.tail()

,fixture_id,date,season,home_team,away_team,home_score,away_score,status,home_shots_on_goal,home_possession,home_passes,home_passes_accuracy,home_fouls,home_corners,home_yellow_cards,home_red_cards,away_shots_on_goal,away_possession,away_passes,away_passes_accuracy,away_fouls,away_corners,away_yellow_cards,away_red_cards,home_last_position,home_last_won,home_last_drawn,home_last_lost,home_last_gf,home_last_ga,home_last_gd,home_last_points,away_last_position,away_last_won,away_last_drawn,away_last_lost,away_last_gf,away_last_ga,away_last_gd,away_last_points
1895,1379344,2026-05-24 15:00:00+00:00,2025,Manchester City,Aston Villa,NaN,NaN,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,21,8,9,72,44,28,71,6,19,9,10,58,51,7,66
1896,1379345,2026-05-24 15:00:00+00:00,2025,Nottingham Forest,Bournemouth,NaN,NaN,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,19,8,11,58,46,12,65,9,15,11,12,58,46,12,56
1897,1379346,2026-05-24 15:00:00+00:00,2025,Sunderland,Chelsea,NaN,NaN,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,11,5,22,64,65,-1,38,4,20,9,9,64,43,21,69
1898,1379347,2026-05-24 15:00:00+00:00,2025,Tottenham,Everton,NaN,NaN,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,11,5,22,64,65,-1,38,13,11,15,12,42,44,-2,48
1899,1379348,2026-05-24 15:00:00+00:00,2025,West Ham,Leeds,NaN,NaN,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,11,10,17,46,62,-16,43,17,11,5,22,64,65,-1,38


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixture_id            1900 non-null   int64  
 1   date                  1900 non-null   object 
 2   season                1900 non-null   int64  
 3   home_team             1900 non-null   object 
 4   away_team             1900 non-null   object 
 5   home_score            1639 non-null   float64
 6   away_score            1639 non-null   float64
 7   status                1900 non-null   object 
 8   home_shots_on_goal    1639 non-null   float64
 9   home_possession       1639 non-null   float64
 10  home_passes           1639 non-null   float64
 11  home_passes_accuracy  1639 non-null   float64
 12  home_fouls            1639 non-null   float64
 13  home_corners          1639 non-null   float64
 14  home_yellow_cards     1639 non-null   float64
 15  home_red_cards       

In [6]:
# データを日付とfixture_idでソート (時系列順に並べるため)
df = df.sort_values(by=['date', 'fixture_id']).reset_index(drop=True)

In [7]:
df["status"].unique()

array(['FT', 'NS'], dtype=object)

# データタイプ変換

In [8]:
#targetカラム作成(試合の勝敗カラム)　0:home win 1:away win 2:draw
def target_create(row):
    if row["home_score"] > row["away_score"]:
        return  "H"    
    
    elif row["home_score"] < row["away_score"]:
        return  "A"  
    
    else:
        return  "D"  
    

df["target"] = df.apply(target_create,axis = 1)

df.head()

    


,fixture_id,date,season,home_team,away_team,home_score,away_score,status,home_shots_on_goal,home_possession,home_passes,home_passes_accuracy,home_fouls,home_corners,home_yellow_cards,home_red_cards,away_shots_on_goal,away_possession,away_passes,away_passes_accuracy,away_fouls,away_corners,away_yellow_cards,away_red_cards,home_last_position,home_last_won,home_last_drawn,home_last_lost,home_last_gf,home_last_ga,home_last_gd,home_last_points,away_last_position,away_last_won,away_last_drawn,away_last_lost,away_last_gf,away_last_ga,away_last_gd,away_last_points,target
0,710556,2021-08-13 19:00:00+00:00,2021,Brentford,Arsenal,2.0,0.0,FT,3.0,35.0,309.0,201.0,12.0,2.0,0.0,0.0,4.0,65.0,568.0,488.0,8.0,5.0,0.0,0.0,17,10,9,19,33,55,-22,39,8,18,7,13,55,39,16,61,H
1,710561,2021-08-14 11:30:00+00:00,2021,Manchester United,Leeds,5.0,1.0,FT,8.0,49.0,422.0,334.0,11.0,5.0,1.0,0.0,3.0,51.0,438.0,343.0,9.0,4.0,2.0,0.0,2,21,11,6,73,44,29,74,9,18,5,15,62,54,8,59,H
2,710557,2021-08-14 14:00:00+00:00,2021,Burnley,Brighton,1.0,2.0,FT,3.0,36.0,259.0,181.0,10.0,7.0,2.0,0.0,8.0,64.0,518.0,424.0,7.0,6.0,1.0,0.0,17,10,9,19,33,55,-22,39,16,9,14,15,40,46,-6,41,A
3,710558,2021-08-14 14:00:00+00:00,2021,Chelsea,Crystal Palace,3.0,0.0,FT,6.0,62.0,678.0,623.0,15.0,5.0,0.0,0.0,1.0,38.0,423.0,363.0,11.0,2.0,0.0,0.0,4,19,10,9,58,36,22,67,14,12,8,18,41,66,-25,44,H
4,710559,2021-08-14 14:00:00+00:00,2021,Everton,Southampton,3.0,1.0,FT,6.0,48.0,337.0,235.0,13.0,6.0,2.0,0.0,3.0,52.0,370.0,256.0,15.0,8.0,0.0,0.0,10,17,8,13,47,48,-1,59,15,12,7,19,47,68,-21,43,H


In [9]:
#objectからcategoryに変換したいカラム
columns = ["home_team","away_team","status","target"]

#タイプ変換関数
def change_type(columns):
    for col in columns:
        df[col] = df[col].astype("category")
    return df

df = change_type(columns) 

In [10]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].dt.tz_localize(None)


# 特徴量エンジニアリング

In [11]:
df["target"].value_counts()

target
H    739
D    636
A    525
Name: count, dtype: int64

In [12]:
# ホーム勝利を示す一時的な列を作成
# 'target'が 'H' の場合に1、それ以外は0
df['is_home_win'] = (df['target'] == 'H').astype(int)
# アウェイ勝利を示す一時的な列を作成
# 'target'が 'Ａ' の場合に1、それ以外は0
df['is_away_win'] = (df['target'] == 'A').astype(int)

# チームごとの直近5試合の勝利数 (is_home_win=1) を計算する関数作成

def home_recent_wins_5(group):
    # 勝利数 (is_home_win) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_team_recent_5_wins'] = group['is_home_win'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_team_recent_5_wins'] = group['home_team_recent_5_wins'].fillna(0)

    # 一時的に作成した列を削除
    return group.drop(columns=['is_home_win'])

def away_recent_wins_5(group):
    # 勝利数 (is_away_win) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_team_recent_5_wins'] = group['is_away_win'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_team_recent_5_wins'] = group['away_team_recent_5_wins'].fillna(0)

    # 一時的に作成した列を削除
    return group.drop(columns=['is_away_win'])

# home_team ごとにグループ化し、calculate_recent_wins_with_target関数を適用します
df = df.groupby('home_team', group_keys=False).apply(home_recent_wins_5)
df = df.groupby('away_team', group_keys=False).apply(away_recent_wins_5)
df["home_team_recent_5_wins"] = df["home_team_recent_5_wins"].astype("int")
df["away_team_recent_5_wins"] = df["away_team_recent_5_wins"].astype("int")

# 新しい特徴量を含むデータフレームの確認
display(df[['date', 'home_team', 'away_team', 'target', 'home_team_recent_5_wins','away_team_recent_5_wins']].head(100))

C:\Users\image\AppData\Local\Temp\ipykernel_17376\1772447025.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby('home_team', group_keys=False).apply(home_recent_wins_5)
C:\Users\image\AppData\Local\Temp\ipykernel_17376\1772447025.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('home_team', group_keys=False).apply(home_recent_wins_5)
C:\Users\image\AppData\Local\Temp\ipykernel_17376\1772447025.py:34: FutureWarning: The default of observed=False is deprecated and will be chang

,date,home_team,away_team,target,home_team_recent_5_wins,away_team_recent_5_wins
0,2021-08-13 19:00:00,Brentford,Arsenal,H,0,0
1,2021-08-14 11:30:00,Manchester United,Leeds,H,0,0
2,2021-08-14 14:00:00,Burnley,Brighton,A,0,0
3,2021-08-14 14:00:00,Chelsea,Crystal Palace,H,0,0
4,2021-08-14 14:00:00,Everton,Southampton,H,0,0
5,2021-08-14 14:00:00,Leicester,Wolves,H,0,0
6,2021-08-14 14:00:00,Watford,Aston Villa,H,0,0
7,2021-08-14 16:30:00,Norwich,Liverpool,A,0,0
8,2021-08-15 13:00:00,Newcastle,West Ham,A,0,0
9,2021-08-15 15:30:00,Tottenham,Manchester City,H,0,0


In [13]:
# 得失点差カラム作成
df["home_goal_difference"] = df["home_score"] - df["away_score"]
df["away_goal_difference"] = df["away_score"] - df["home_score"]

df.head()


,fixture_id,date,season,home_team,away_team,home_score,away_score,status,home_shots_on_goal,home_possession,home_passes,home_passes_accuracy,home_fouls,home_corners,home_yellow_cards,home_red_cards,away_shots_on_goal,away_possession,away_passes,away_passes_accuracy,away_fouls,away_corners,away_yellow_cards,away_red_cards,home_last_position,home_last_won,home_last_drawn,home_last_lost,home_last_gf,home_last_ga,home_last_gd,home_last_points,away_last_position,away_last_won,away_last_drawn,away_last_lost,away_last_gf,away_last_ga,away_last_gd,away_last_points,target,home_team_recent_5_wins,away_team_recent_5_wins,home_goal_difference,away_goal_difference
0,710556,2021-08-13 19:00:00,2021,Brentford,Arsenal,2.0,0.0,FT,3.0,35.0,309.0,201.0,12.0,2.0,0.0,0.0,4.0,65.0,568.0,488.0,8.0,5.0,0.0,0.0,17,10,9,19,33,55,-22,39,8,18,7,13,55,39,16,61,H,0,0,2.0,-2.0
1,710561,2021-08-14 11:30:00,2021,Manchester United,Leeds,5.0,1.0,FT,8.0,49.0,422.0,334.0,11.0,5.0,1.0,0.0,3.0,51.0,438.0,343.0,9.0,4.0,2.0,0.0,2,21,11,6,73,44,29,74,9,18,5,15,62,54,8,59,H,0,0,4.0,-4.0
2,710557,2021-08-14 14:00:00,2021,Burnley,Brighton,1.0,2.0,FT,3.0,36.0,259.0,181.0,10.0,7.0,2.0,0.0,8.0,64.0,518.0,424.0,7.0,6.0,1.0,0.0,17,10,9,19,33,55,-22,39,16,9,14,15,40,46,-6,41,A,0,0,-1.0,1.0
3,710558,2021-08-14 14:00:00,2021,Chelsea,Crystal Palace,3.0,0.0,FT,6.0,62.0,678.0,623.0,15.0,5.0,0.0,0.0,1.0,38.0,423.0,363.0,11.0,2.0,0.0,0.0,4,19,10,9,58,36,22,67,14,12,8,18,41,66,-25,44,H,0,0,3.0,-3.0
4,710559,2021-08-14 14:00:00,2021,Everton,Southampton,3.0,1.0,FT,6.0,48.0,337.0,235.0,13.0,6.0,2.0,0.0,3.0,52.0,370.0,256.0,15.0,8.0,0.0,0.0,10,17,8,13,47,48,-1,59,15,12,7,19,47,68,-21,43,H,0,0,2.0,-2.0


## ゴール数に関する関数作成

In [14]:
#------------直近5試合のデータ--------------

def home_recent_score_5(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_5_scores'] = group['home_score'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_5_scores'] = group['home_recent_5_scores'].fillna(0)

    return group

def away_recent_score_5(group):
    # ゴール数 (away_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_5_scores'] = group['away_score'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_5_scores'] = group['away_recent_5_scores'].fillna(0)

    return group

def home_recent_goal_against_5(group):
    # 失点数 (away_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_5_goal_against'] = group['away_score'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_5_goal_against'] = group['home_recent_5_goal_against'].fillna(0)

    return group

def away_recent_goal_against_5(group):
    # 失点数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_5_goal_against'] = group['home_score'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_5_goal_against'] = group['away_recent_5_goal_against'].fillna(0)

    return group


def home_recent_goal_diff_5(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_5_goal_diff'] = group['home_goal_difference'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_5_goal_diff'] = group['home_recent_5_goal_diff'].fillna(0)

    return group

def away_recent_goal_diff_5(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_5_goal_diff'] = group['away_goal_difference'].rolling(window=6, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_5_goal_diff'] = group['away_recent_5_goal_diff'].fillna(0)

    return group




    
#-----------------------------直近10試合のデータ-----------------------------------

def home_recent_score_10(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_10_scores'] = group['home_score'].rolling(window=11, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_10_scores'] = group['home_recent_10_scores'].fillna(0)

    return group

def away_recent_score_10(group):
    # ゴール数 (away_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_10_scores'] = group['away_score'].rolling(window=11, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_10_scores'] = group['away_recent_10_scores'].fillna(0)

    return group

def home_recent_goal_against_10(group):
    # 失点数 (away_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_10_goal_against'] = group['away_score'].rolling(window=11, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_10_goal_against'] = group['home_recent_10_goal_against'].fillna(0)

    return group

def away_recent_goal_against_10(group):
    # 失点数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_10_goal_against'] = group['home_score'].rolling(window=11, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_10_goal_against'] = group['away_recent_10_goal_against'].fillna(0)

    return group


def home_recent_goal_diff_10(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_10_goal_diff'] = group['home_goal_difference'].rolling(window=11, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_10_goal_diff'] = group['home_recent_10_goal_diff'].fillna(0)

    return group

def away_recent_goal_diff_10(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_10_goal_diff'] = group['away_goal_difference'].rolling(window=11, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_10_goal_diff'] = group['away_recent_10_goal_diff'].fillna(0)

    return group


#----------------------直近20試合のデータ------------------------



def home_recent_score_20(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_20_scores'] = group['home_score'].rolling(window=21, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_20_scores'] = group['home_recent_20_scores'].fillna(0)

    return group

def away_recent_score_20(group):
    # ゴール数 (away_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_20_scores'] = group['away_score'].rolling(window=21, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_20_scores'] = group['away_recent_20_scores'].fillna(0)

    return group

def home_recent_goal_against_20(group):
    # 失点数 (away_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_20_goal_against'] = group['away_score'].rolling(window=21, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_20_goal_against'] = group['home_recent_20_goal_against'].fillna(0)

    return group

def away_recent_goal_against_20(group):
    # 失点数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_20_goal_against'] = group['home_score'].rolling(window=21, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_20_goal_against'] = group['away_recent_20_goal_against'].fillna(0)

    return group


def home_recent_goal_diff_20(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['home_recent_20_goal_diff'] = group['home_goal_difference'].rolling(window=21, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_recent_20_goal_diff'] = group['home_recent_20_goal_diff'].fillna(0)

    return group

def away_recent_goal_diff_20(group):
    # ゴール数 (home_score) のローリング合計を計算し、1つシフト (現在の試合結果を除く)
    # shift(1) で現在の試合結果を除外し、直前5試合の結果のみを参照
    group['away_recent_20_goal_diff'] = group['away_goal_difference'].rolling(window=21, min_periods=1).sum().shift(1)

    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_recent_20_goal_diff'] = group['away_recent_20_goal_diff'].fillna(0)

    return group


In [15]:
df = df.groupby('home_team', group_keys=False).apply(home_recent_score_5)
df = df.groupby('away_team', group_keys=False).apply(away_recent_score_5)
df = df.groupby('home_team', group_keys=False).apply(home_recent_goal_diff_5)
df = df.groupby('away_team', group_keys=False).apply(away_recent_goal_diff_5)
df = df.groupby('home_team', group_keys=False).apply(home_recent_goal_against_5)
df = df.groupby('away_team', group_keys=False).apply(away_recent_goal_against_5)

df = df.groupby('home_team', group_keys=False).apply(home_recent_score_10)
df = df.groupby('away_team', group_keys=False).apply(away_recent_score_10)
df = df.groupby('home_team', group_keys=False).apply(home_recent_goal_diff_10)
df = df.groupby('away_team', group_keys=False).apply(away_recent_goal_diff_10)
df = df.groupby('home_team', group_keys=False).apply(home_recent_goal_against_10)
df = df.groupby('away_team', group_keys=False).apply(away_recent_goal_against_10)

df = df.groupby('home_team', group_keys=False).apply(home_recent_score_20)
df = df.groupby('away_team', group_keys=False).apply(away_recent_score_20)
df = df.groupby('home_team', group_keys=False).apply(home_recent_goal_diff_20)
df = df.groupby('away_team', group_keys=False).apply(away_recent_goal_diff_20)
df = df.groupby('home_team', group_keys=False).apply(home_recent_goal_against_20)
df = df.groupby('away_team', group_keys=False).apply(away_recent_goal_against_20)


df["home_recent_5_scores"] = df["home_recent_5_scores"].astype("int")
df["away_recent_5_scores"] = df["away_recent_5_scores"].astype("int")
df["home_recent_5_goal_diff"] = df["home_recent_5_goal_diff"].astype("int")
df["away_recent_5_goal_diff"] = df["away_recent_5_goal_diff"].astype("int")
df["home_recent_5_goal_against"] = df["home_recent_5_goal_against"].astype("int")
df["away_recent_5_goal_against"] = df["away_recent_5_goal_against"].astype("int")

df["home_recent_10_scores"] = df["home_recent_10_scores"].astype("int")
df["away_recent_10_scores"] = df["away_recent_10_scores"].astype("int")
df["home_recent_10_goal_diff"] = df["home_recent_10_goal_diff"].astype("int")
df["away_recent_10_goal_diff"] = df["away_recent_10_goal_diff"].astype("int")
df["home_recent_10_goal_against"] = df["home_recent_10_goal_against"].astype("int")
df["away_recent_10_goal_against"] = df["away_recent_10_goal_against"].astype("int")

df["home_recent_20_scores"] = df["home_recent_20_scores"].astype("int")
df["away_recent_20_scores"] = df["away_recent_20_scores"].astype("int")
df["home_recent_20_goal_diff"] = df["home_recent_20_goal_diff"].astype("int")
df["away_recent_20_goal_diff"] = df["away_recent_20_goal_diff"].astype("int")
df["home_recent_20_goal_against"] = df["home_recent_20_goal_against"].astype("int")
df["away_recent_20_goal_against"] = df["away_recent_20_goal_against"].astype("int")

display(df[['date', 'home_team', 'away_team', 'target', 'home_recent_20_scores','away_recent_20_scores','home_recent_20_goal_against',
            'away_recent_20_goal_against','home_recent_20_goal_diff','away_recent_20_goal_diff', 'home_recent_10_scores','away_recent_10_scores',
            'home_recent_10_goal_against','away_recent_10_goal_against','home_recent_10_goal_diff','away_recent_10_goal_diff']].tail(50))

C:\Users\image\AppData\Local\Temp\ipykernel_17376\720431589.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby('home_team', group_keys=False).apply(home_recent_score_5)
C:\Users\image\AppData\Local\Temp\ipykernel_17376\720431589.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('home_team', group_keys=False).apply(home_recent_score_5)
C:\Users\image\AppData\Local\Temp\ipykernel_17376\720431589.py:2: FutureWarning: The default of observed=False is deprecated and will be changed t

,date,home_team,away_team,target,home_recent_20_scores,away_recent_20_scores,home_recent_20_goal_against,away_recent_20_goal_against,home_recent_20_goal_diff,away_recent_20_goal_diff,home_recent_10_scores,away_recent_10_scores,home_recent_10_goal_against,away_recent_10_goal_against,home_recent_10_goal_diff,away_recent_10_goal_diff
1850,2026-04-25 14:00:00,Arsenal,Newcastle,D,23,9,8,14,15,-5,4,1,1,3,3,-2
1851,2026-04-25 14:00:00,Bournemouth,Leeds,D,15,9,8,30,7,-21,2,1,2,3,0,-2
1852,2026-04-25 14:00:00,Brighton,Chelsea,D,21,17,16,9,5,8,2,2,1,0,1,2
1853,2026-04-25 14:00:00,Burnley,Manchester City,D,11,12,15,7,-4,5,0,1,2,2,-2,-1
1854,2026-04-25 14:00:00,Fulham,Aston Villa,D,16,13,12,10,4,3,1,2,0,1,1,1
1855,2026-04-25 14:00:00,Liverpool,Crystal Palace,D,21,15,13,17,8,-2,0,2,3,0,-3,2
1856,2026-04-25 14:00:00,Manchester United,Brentford,D,14,12,10,16,4,-4,0,1,0,2,0,-1
1857,2026-04-25 14:00:00,Sunderland,Nottingham Forest,D,11,15,5,16,6,-1,2,3,2,0,0,3
1858,2026-04-25 14:00:00,West Ham,Everton,D,13,10,22,11,-9,-1,3,0,2,0,1,0
1859,2026-04-25 14:00:00,Wolves,Tottenham,D,15,17,21,20,-6,-3,0,1,2,4,-2,-3


## 勝ち点カラム作成

In [16]:
#home,awayの区別なしのデータを作成する
#home,awayの区別をしないカラムを作成する場合には、ここで作成した"df_stacked"をベースに作成していく

# 各試合に一意のIDを付与 (後のマージのために利用)
df['match_id'] = df.index 

# 2. チーム視点でのデータ作成 (スタッキング)
# -----------------------------------------------

# ① ホームチーム視点のデータフレームを作成
df_home = df[['match_id','season','date', 'home_team', 'target']].copy()
df_home.rename(columns={'home_team': 'team'}, inplace=True)
# ホームチーム視点での勝利: targetが'H'のとき1(勝ち点カウント用)
df_home['points'] = df_home['target'].map({"H":3,"D":1,"A":0})

# ホームチーム視点での勝利: targetが'H'のとき1　（勝率計算用）　
df_home['is_win'] = (df_home['target'] == 'H').astype(int)

# # ② アウェイチーム視点のデータフレームを作成
df_away = df[['match_id','season', 'date', 'away_team', 'target']].copy()
df_away.rename(columns={'away_team': 'team'}, inplace=True)

# アウェイチーム視点での勝利: targetが'A'のとき1
df_away['points'] = df_home['target'].map({"H":0,"D":1,"A":3})

# アウェイチーム視点での勝利: targetが'A'のとき1
df_away['is_win'] = (df_away['target'] == 'A').astype(int)

# # ③ 2つの視点のデータを結合し、時系列順にソート
df_stacked = pd.concat([df_home, df_away],ignore_index=True)
df_stacked = df_stacked.sort_values(by=['date', 'match_id']).reset_index(drop=True)


# display(df_stacked_point.head(20))
# print(df_stacked_point.info())


In [ ]:
# seasonとteamでグループ化して、各シーズン内で、各チームの勝ち点をrolling_windowを使用して、累積していく。
#それにより、各チームの各試合前の時点での合計勝ち点を計算して、新たなカラムに格納する

#home team
def calculate_home_points(group):
    group["home_total_points"] = group["points"].rolling(window = 38,min_periods = 1).sum().shift(1)

    group["home_total_points"] = group["home_total_points"].fillna(0)

    return group
    
#away team
def calculate_away_points(group):
    group["away_total_points"] = group["points"].rolling(window = 38,min_periods = 1).sum().shift(1)

    group["away_total_points"] = group["away_total_points"].fillna(0)

    return group


df_stacked = df_stacked.groupby(["season","team"], group_keys=False).apply(calculate_home_points)
df_stacked = df_stacked.groupby(["season","team"], group_keys=False).apply(calculate_away_points)


# ホームチームの総合成績フィーチャーのみを抽出 (match_id と team で紐づけ)
df_home = df_stacked[['match_id','team','home_total_points']]
df_home.rename(columns={'team': 'home_team'}, inplace=True)

df = pd.merge(
    df,
    df_home,
    on = ['match_id','home_team'],
    how = 'left'
)
#アウェイチーム
df_away = df_stacked[['match_id','team','away_total_points']]
df_away.rename(columns={'team': 'away_team'}, inplace=True)

df = pd.merge(
    df,
    df_away,
    on = ['match_id','away_team'],
    how = 'left'
)

df['home_total_points'] = df['home_total_points'].astype(int)
df['away_total_points'] = df['away_total_points'].astype(int)


# home teamとaway teamの勝ち点差カラムを作成
df["points_difference"] = df['home_total_points'] - df['away_total_points']

df[['match_id','season','date','home_team','away_team','target','home_total_points','away_total_points','points_difference']]

## 各チームの直近5試合での勝利数カラム作成

In [ ]:
#  ローリング計算の実行関数作成
# ----------------------------
#groupbyによって、各チームごとの全試合のデータを集めたグループを作成し、そのグループ（各チームの試合だけをまとめたデータフレーム）ごとに、rollingにより、その試合を含む直近6試合の勝利数をカウント。
#shift(1)によって、1試合前までの5試合での合計勝利数を計算する

#-------------------home team------------------------

def home_recent_5_wins_overall(group):
    group['home_team_recent_5_wins_overall'] = group['is_win'].rolling(window=6, min_periods=1).sum().shift(1)
    
    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['home_team_recent_5_wins_overall'] = group['home_team_recent_5_wins_overall'].fillna(0)

    return group

df_stacked = df_stacked.groupby(['season','team'], group_keys=False).apply(home_recent_5_wins_overall)

# ホームチームの総合成績フィーチャーのみを抽出 (match_id と team で紐づけ)
df_home_feature = df_stacked[['match_id', 'team', 'home_team_recent_5_wins_overall']].copy()
df_home_feature.rename(columns={'team': 'home_team'}, inplace=True)

# 元のデータフレームにマージ
df = pd.merge(
    df, 
    df_home_feature, 
    on=['match_id', 'home_team'], 
    how='left'
)


#------------------away team--------------------

def away_recent_5_wins_overall(group):
    group['away_team_recent_5_wins_overall'] = group['is_win'].rolling(window=6, min_periods=1).sum().shift(1)
    
    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_team_recent_5_wins_overall'] = group['home_team_recent_5_wins_overall'].fillna(0)

    return group

df_stacked = df_stacked.groupby(['season','team'], group_keys=False).apply(away_recent_5_wins_overall)

# ホームチームの総合成績フィーチャーのみを抽出 (match_id と team で紐づけ)
df_away_feature = df_stacked[['match_id', 'team', 'away_team_recent_5_wins_overall']].copy()
df_away_feature.rename(columns={'team': 'away_team'}, inplace=True)

# 元のデータフレームにマージ
df = pd.merge(
    df, 
    df_away_feature, 
    on=['match_id', 'away_team'], 
    how='left'
)

# 新しい特徴量を含むデータフレームの確認
display(df[['date', 'home_team', 'away_team', 'target', 'home_team_recent_5_wins_overall','away_team_recent_5_wins_overall']])

print(df.info())

In [ ]:
#  ローリング計算の実行関数作成
# ----------------------------
#groupbyによって、各チームごとの全試合のデータを集めたグループを作成し、そのグループ（各チームの試合だけをまとめたデータフレーム）ごとに、rollingにより、その試合を含む直近6試合の勝率を計算。
#shift(1)によって、1試合前までの5試合での勝率を計算する

#-------------------home team------------------------

def home_season_wins_ave_overall(group):
    group['home_season_wins_ave_overall'] = (group['is_win'].rolling(window=38, min_periods=1).mean().shift(1)*100).round(2)
    
    # 初回試合などで直近の試合がない場合 の処理は要検討
    group['home_season_wins_ave_overall'] = group['home_season_wins_ave_overall'].fillna(0)

    return group

df_stacked = df_stacked.groupby(['season','team'], group_keys=False).apply(home_season_wins_ave_overall)

# ホームチームの総合成績フィーチャーのみを抽出 (match_id と team で紐づけ)
df_home_ave = df_stacked[['match_id', 'team', 'home_season_wins_ave_overall']].copy()
df_home_ave.rename(columns={'team': 'home_team'}, inplace=True)

# 元のデータフレームにマージ
df = pd.merge(
    df, 
    df_home_ave, 
    on=['match_id', 'home_team'], 
    how='left'
)


# #------------------away team--------------------

def away_season_wins_ave_overall(group):
    group['away_season_wins_ave_overall'] = (group['is_win'].rolling(window=6, min_periods=1).mean().shift(1)*100).round(2)
    
    # 初回試合などで直近の試合がない場合 (NaN) は 0 で埋める
    group['away_season_wins_ave_overall'] = group['away_season_wins_ave_overall'].fillna(0)

    return group

df_stacked = df_stacked.groupby(['season','team'], group_keys=False).apply(away_season_wins_ave_overall)

# ホームチームの総合成績フィーチャーのみを抽出 (match_id と team で紐づけ)
df_away_feature = df_stacked[['match_id', 'team', 'away_season_wins_ave_overall']].copy()
df_away_feature.rename(columns={'team': 'away_team'}, inplace=True)

# 元のデータフレームにマージ
df = pd.merge(
    df, 
    df_away_feature, 
    on=['match_id', 'away_team'], 
    how='left'
)

# 新しい特徴量を含むデータフレームの確認
display(df[['date','season','home_team', 'away_team', 'target', 'home_season_wins_ave_overall', 'away_season_wins_ave_overall']].tail(50))

print(df.info())

In [ ]:
df[df["status"] == "FT"].tail(50)

In [ ]:
df[df["status"] == "NS"].head(2)

In [ ]:
df.columns

In [ ]:
fill_features_home = ['home_team_recent_5_wins',
                    'home_recent_5_scores', 
                    'home_recent_5_goal_diff', 
                    'home_recent_5_goal_against', 
                    'home_recent_10_scores', 
                    'home_recent_10_goal_diff', 
                    'home_recent_10_goal_against', 
                    'home_recent_20_scores', 
                    'home_recent_20_goal_diff', 
                    'home_recent_20_goal_against']


fill_features_away = ['away_team_recent_5_wins',
                    'away_recent_5_scores',
                    'away_recent_5_goal_diff',
                    'away_recent_5_goal_against',
                    'away_recent_10_scores',
                    'away_recent_10_goal_diff',
                    'away_recent_10_goal_against',
                    'away_recent_20_scores',
                    'away_recent_20_goal_diff',
                    'away_recent_20_goal_against']

fill_features_home_overall = ['home_total_points','home_team_recent_5_wins_overall','home_season_wins_ave_overall']

fill_features_away_overall = ['away_total_points','away_team_recent_5_wins_overall','away_season_wins_ave_overall']

In [ ]:
fill_features =  ['home_team_recent_5_wins',
       'away_team_recent_5_wins', 'home_goal_difference',
       'away_goal_difference', 'home_recent_5_scores', 'away_recent_5_scores',
       'home_recent_5_goal_diff', 'away_recent_5_goal_diff',
       'home_recent_5_goal_against', 'away_recent_5_goal_against',
       'home_recent_10_scores', 'away_recent_10_scores',
       'home_recent_10_goal_diff', 'away_recent_10_goal_diff',
       'home_recent_10_goal_against', 'away_recent_10_goal_against',
       'home_recent_20_scores', 'away_recent_20_scores',
       'home_recent_20_goal_diff', 'away_recent_20_goal_diff',
       'home_recent_20_goal_against', 'away_recent_20_goal_against',
       'match_id', 'home_total_points', 'away_total_points',
       'points_difference', 'home_team_recent_5_wins_overall',
       'away_team_recent_5_wins_overall', 'home_season_wins_ave_overall',
       'away_season_wins_ave_overall']

In [ ]:
teams = df[df["season"] == 2025]["home_team"].unique()

In [ ]:
teams

In [ ]:
#statusがNSの試合のデータの欠損値を各チームごとに一番最後に行われた試合のデータで補充する

#home限定データの補充(homeでの試合のみでカウントしているデータ)
for team in teams:
    # 置き換え元（FT & Chelsea）の最後の行の fill_features を取得
    source_vals = df[(df["status"] == "FT") & (df["home_team"] == team)].iloc[-1][fill_features_home]
    
    # 置き換え先（NS & Chelsea）に代入
    df.loc[(df["status"] == "NS") & (df["home_team"] == team), fill_features_home] = source_vals.values

#away限定データの補充(awayでの試合のみでカウントしているデータ)
for team in teams:
    # 置き換え元（FT）の最後の行の fill_features を取得
    source_vals = df[(df["status"] == "FT") & (df["away_team"] == team)].iloc[-1][fill_features_away]
    
    # 置き換え先（NS）に代入
    df.loc[(df["status"] == "NS") & (df["away_team"] == team), fill_features_away] = source_vals.values

#home,awayの区別なしでカウントしているデータの補充
for team in teams:
    # 最新試合を日付順で取得
    source_df = df[(df["status"] == "FT") & ((df["home_team"] == team) | (df["away_team"] == team))].sort_values("date")
    if source_df.empty:
        continue

    last_row = source_df.iloc[-1]

    # 最新試合の home/away 両方の overall 値を取得
    latest_home_vals = last_row[fill_features_home_overall].values
    latest_away_vals = last_row[fill_features_away_overall].values

    # NS の行ごとに index を取得
    ns_home_idx = df[(df["status"] == "NS") & (df["home_team"] == team)].index
    ns_away_idx = df[(df["status"] == "NS") & (df["away_team"] == team)].index

    # 代入
    df.loc[ns_home_idx, fill_features_home_overall] = [latest_home_vals] * len(ns_home_idx)
    df.loc[ns_away_idx, fill_features_away_overall] = [latest_away_vals] * len(ns_away_idx)



In [ ]:
for team in teams:
    print("team",team)
    display(df.loc[(df["status"] == "FT") & (df["home_team"] == team), fill_features_home].tail(1))
    display(df.loc[(df["status"] == "NS") & (df["home_team"] == team), fill_features_home].head(2))
    print()
    display(df.loc[(df["status"] == "FT") & (df["away_team"] == team), fill_features_away].tail(1))
    display(df.loc[(df["status"] == "NS") & (df["away_team"] == team), fill_features_away].head(2))
    print()
    
    source_df = df[(df["status"] == "FT") & ((df["home_team"] == team) | (df["away_team"] == team))].sort_values("date")
    if source_df.empty:
        continue
    
    last_row = source_df.iloc[-1]

    # 2. 対象チームが最新試合でホームだったかアウェイだったかを判別
    team_was_home = last_row["home_team"] == team
    
    # 3. 最新試合の行から、対象チーム自身の統計値のみを1セット取得
    if team_was_home:
        # チームがホームの場合、統計値は home_overall カラムにある
        display(last_row[fill_features_home_overall])
        display(df.loc[(df["status"] == "NS") & (df["home_team"] == team), fill_features_home_overall].head(2))
        display(df.loc[(df["status"] == "NS") & (df["away_team"] == team), fill_features_away_overall].head(2))
    else:
        display(last_row[fill_features_away_overall])
        display(df.loc[(df["status"] == "NS") & (df["home_team"] == team), fill_features_home_overall].head(2))
        display(df.loc[(df["status"] == "NS") & (df["away_team"] == team), fill_features_away_overall].head(2))

    
    
    

#　データセット作成

In [ ]:
# train_val_df = df[df["date"] < "2024-03-01"]
# test_df = df[df["date"] >= "2024-03-01"]

train_val_df = df[df["status"] == "FT"]
test_df = df[df["status"] == "NS"]

In [ ]:
train_val_df[train_val_df["season"] == 2025][["date","home_team","away_team","target"]]

test_df.head()

# モデル学習に使用する特徴量の設定 & 説明変数と目的変数にデータを分割

In [ ]:
#現在保有している特徴量名を確認
train_val_df.columns

In [ ]:
features = ["home_team","away_team",'home_season_wins_ave_overall',
           'away_season_wins_ave_overall','home_last_points','away_last_points']

target = "target"

In [ ]:
x_all = train_val_df[features]
y_all = train_val_df[target]

print(x_all.shape)
print(y_all.shape)

# 学習データと検証データの期間設定



In [ ]:
# folds = [
#     {"train_end": "2023-9-30", "val_start": "2023-10-01", "val_end": "2023-12-31"},
#     {"train_end": "2023-10-31", "val_start": "2023-11-01", "val_end": "2024-01-31"},
#     {"train_end": "2023-11-30", "val_start": "2023-12-01", "val_end": "2024-02-29"},
# ]

folds = [
    {"train_end": "2025-9-20", "val_start": "2025-9-21", "val_end": "2025-10-20"},
    {"train_end": "2025-9-30", "val_start": "2025-10-01", "val_end": "2025-10-31"},
    {"train_end": "2025-10-10", "val_start": "2025-10-11", "val_end": "2025-11-10"},
]

In [ ]:
#ハイパーパラメータの設定
params = {
    "n_estimators":1000,
    "learning_rate":0.05,
    "num_leaves":32
}

# モデル学習&評価関数

In [ ]:
#訓練データと検証データのindex作成
# foldsの中から、今回設定した範囲を取り出し、その範囲に入っているかどうかを判断し、その範囲内のデータのみを訓練データと検証データとしていく。
#これを3周する


def train_lgb(original_df,
              input_x,
              input_y,
              list_nfold=[0,1,2],
              params=params,
              folds=folds
              ):

    #推論値を格納する変数の作成
    df_valid_pred = pd.DataFrame()
    #評価値を入れる変数の作成
    metrics_tr = [] #学習データ用
    metrics_val = [] #検証データ用
    #重要度を格納するデータフレームの作成
    df_imp = pd.DataFrame()
    #レポートを入れる変数の作成
    report_tr_list = []
    report_val_list = []
    

    for i,nfold in enumerate(list_nfold):
        print("fold:",i)
        # 学習用
        train_idx = original_df["date"] <= folds[nfold]["train_end"]
        x_tr = input_x[train_idx]
        y_tr = input_y[train_idx]
        
        # 検証用
        val_idx = (original_df["date"] >= folds[nfold]["val_start"]) & (original_df["date"] <= folds[nfold]["val_end"])
        x_val = input_x[val_idx]
        y_val = input_y[val_idx]

        # LightGBM モデル
        model = lgb.LGBMClassifier(**params)
        
        
        # モデルの訓練
        model.fit(
            x_tr, y_tr,
            eval_set=[(x_val, y_val)],
            eval_metric="multi_logloss",
            callbacks=[
            early_stopping(stopping_rounds=50),  # 早期停止
            log_evaluation(50)                   # 50回ごとにログ表示
            ]
            )
        #保存するモデルのファイル名
        filename = "model_lgb_fold{}.pickle".format(nfold)
        
        #モデルを保存
        with open(filename, 'wb') as f:
            pickle.dump(model, f, protocol=4)




#-------------------学習データの評価のコード-----------------------

        acc_tr,ll_tr,f1_macro_tr,f1_weighted_tr,report_tr,y_pred_tr,y_pred_proba_tr = evaluate_model(model,x_tr,y_tr)    
        
        print(f"Fold {i+1} ACC: {acc_tr:.4f}, LogLoss: {ll_tr:.4f}, F1(macro): {f1_macro_tr:.4f}, F1(weighted): {f1_weighted_tr:.4f}")

        #検証スコアを各foldごとに格納 #あとでデータフレーム型に変更
        metrics_tr.append({
            "nfold": nfold,
            "accuracy": acc_tr,
            "log_loss": ll_tr,
            "f1_macro": f1_macro_tr,
            "f1_weighted": f1_weighted_tr,
            # per-class f1 を追加
            "f1_A": report_tr["A"]["f1-score"] if "A" in report_tr else None,
            "f1_D": report_tr["D"]["f1-score"] if "D" in report_tr else None,
            "f1_H": report_tr["H"]["f1-score"] if "H" in report_tr else None,
        })

        report_tr_list.append({
            "fold": nfold,
            "report": report_tr
            })


        
#----------------------以下、検証データの評価のコード---------------------------------

        acc_val,ll_val,f1_macro_val,f1_weighted_val,report_val,y_pred_val,y_pred_proba_val = evaluate_model(model,x_val,y_val)    

        print(f"Fold {i+1} ACC: {acc_val:.4f}, LogLoss: {ll_val:.4f}, F1(macro): {f1_macro_val:.4f}, F1(weighted): {f1_weighted_val:.4f}")

        #検証スコアを各foldごとに格納 #あとでデータフレーム型に変更
        metrics_val.append({
            "nfold": nfold,
            "accuracy": acc_val,
            "log_loss": ll_val,
            "f1_macro": f1_macro_val,
            "f1_weighted": f1_weighted_val,
            # per-class f1 を追加
            "f1_A": report_val["A"]["f1-score"] if "A" in report_val else None,
            "f1_D": report_val["D"]["f1-score"] if "D" in report_val else None,
            "f1_H": report_val["H"]["f1-score"] if "H" in report_val else None,
        })

        report_val_list.append({
            "fold": nfold,
            "report": report_val
            })



#----------------検証データの予測値の格納と特徴量重要度の格納---------------------



        #各foldごとに実際の値と予測値を格納したDataFrameを作成
        
        df_pred = pd.DataFrame({
            "nfold": [nfold] * len(y_val),
            "true": y_val,
            "pred": y_pred_val,
            "prob_A": y_pred_proba_val[:, 0],
            "prob_D": y_pred_proba_val[:, 1],
            "prob_H": y_pred_proba_val[:, 2]
        })


        # df_pred = pd.DataFrame({"nfold":nfold,"true":y_val,"pred_proba":y_pred_proba,"pred":y_pred})
        df_valid_pred = pd.concat([df_valid_pred,df_pred],axis = 0,ignore_index=True)
        
        #各foldごとに各変数の重要度を格納したデータフレームを作成
        tmp_imp = pd.DataFrame({"nfold":nfold,"col":x_tr.columns,"imp":model.feature_importances_})
        df_imp = pd.concat([df_imp,tmp_imp])

#------------------------最終処理---------------------------
    
    print("-"*10,"result","-"*10)
    
    #各foldごとの評価値をデータフレームに格納
    df_metrics_tr = pd.DataFrame(metrics_tr,columns = ["nfold","accuracy","log_loss","f1_macro","f1_weighted","f1_A","f1_D","f1_H"]) #学習データ評価値
    df_metrics_val = pd.DataFrame(metrics_val,columns = ["nfold","accuracy","log_loss","f1_macro","f1_weighted","f1_A","f1_D","f1_H"]) #検証データ評価値

    print("-"*10,"学習データ","-"*10)
    print("各foldのaccuracyの平均:{:.2f}".format(df_metrics_tr['accuracy'].mean()))
    print("各foldのlog_lossの平均:{:.2f}".format(df_metrics_tr['log_loss'].mean()))
    print("各foldのf1_macroの平均:{:.2f}".format(df_metrics_tr['f1_macro'].mean()))
    print("各foldのf1_weightedの平均:{:.2f}".format(df_metrics_tr['f1_weighted'].mean()))
    print("各foldのf1_Hの平均:{:.2f}".format(df_metrics_tr['f1_A'].mean()))
    print("各foldのf1_Aの平均:{:.2f}".format(df_metrics_tr['f1_D'].mean()))
    print("各foldのf1_Dの平均:{:.2f}".format(df_metrics_tr['f1_H'].mean()))
    print()
    print("-"*10,"検証データ","-"*10)
    print("各foldのaccuracyの平均:{:.2f}".format(df_metrics_val['accuracy'].mean()))
    print("各foldのlog_lossの平均:{:.2f}".format(df_metrics_val['log_loss'].mean()))
    print("各foldのf1_macroの平均:{:.2f}".format(df_metrics_val['f1_macro'].mean()))
    print("各foldのf1_weightedの平均:{:.2f}".format(df_metrics_val['f1_weighted'].mean()))
    print("各foldのf1_Hの平均:{:.2f}".format(df_metrics_val['f1_A'].mean()))
    print("各foldのf1_Aの平均:{:.2f}".format(df_metrics_val['f1_D'].mean()))
    print("各foldのf1_Dの平均:{:.2f}".format(df_metrics_val['f1_H'].mean()))


    return df_valid_pred,df_imp,df_metrics_tr,df_metrics_val,report_tr_list,report_val_list

# モデル学習&評価関数内に組み込む各種評価値の算出用関数

In [ ]:
def evaluate_model(model, X, y):
    y_pred_proba = model.predict_proba(X)
    pred_idx = np.argmax(y_pred_proba, axis=1)
    y_pred = model.classes_[pred_idx]

    acc = accuracy_score(y, y_pred)
    ll = log_loss(y, y_pred_proba, labels=model.classes_)
    f1_macro = f1_score(y, y_pred, average="macro")
    f1_weighted = f1_score(y, y_pred, average="weighted")
    report = classification_report(y, y_pred, labels=model.classes_, output_dict=True)

    return acc, ll, f1_macro, f1_weighted, report, y_pred, y_pred_proba


In [ ]:
df_valid_pred,df_imp,df_metrics_tr,df_metrics_val,report_tr,report_val = train_lgb(train_val_df,
                                            x_all,
                                            y_all,
                                            list_nfold=[0,1,2],
                                            params=params,
                                            folds=folds
                                            )

In [ ]:
filename = "model_lgb_fold1.pickle" #学習したモデルの読み込み
with open(filename,"rb") as f:
    model = pickle.load(f)


# モデル評価値

In [ ]:
#各変数ごとに3回のfoldでの重要度の平均を算出
print("モデルの特徴量の重要度")
df_imp_mean = df_imp.groupby("col")["imp"].agg(["mean","std"]).sort_values("mean",ascending=False)
df_imp_mean

In [ ]:
print("学習データスコア表")
display(df_metrics_tr)
print()
print("検証データスコア表")
display(df_metrics_val)

print("各foldのaccuracyの平均:{:.2f}".format(df_metrics_tr['accuracy'].mean()))
print("各foldのlog_lossの平均:{:.2f}".format(df_metrics_tr['log_loss'].mean()))
print("各foldのf1_macroの平均:{:.2f}".format(df_metrics_tr['f1_macro'].mean()))
print("各foldのf1_weightedの平均:{:.2f}".format(df_metrics_tr['f1_weighted'].mean()))
print("各foldのf1_Hの平均:{:.2f}".format(df_metrics_tr['f1_A'].mean()))
print("各foldのf1_Aの平均:{:.2f}".format(df_metrics_tr['f1_D'].mean()))
print("各foldのf1_Dの平均:{:.2f}".format(df_metrics_tr['f1_H'].mean()))
print()
print("-"*10,"検証データ","-"*10)
print("各foldのaccuracyの平均:{:.2f}".format(df_metrics_val['accuracy'].mean()))
print("各foldのlog_lossの平均:{:.2f}".format(df_metrics_val['log_loss'].mean()))
print("各foldのf1_macroの平均:{:.2f}".format(df_metrics_val['f1_macro'].mean()))
print("各foldのf1_weightedの平均:{:.2f}".format(df_metrics_val['f1_weighted'].mean()))
print("各foldのf1_Hの平均:{:.2f}".format(df_metrics_val['f1_A'].mean()))
print("各foldのf1_Aの平均:{:.2f}".format(df_metrics_val['f1_D'].mean()))
print("各foldのf1_Dの平均:{:.2f}".format(df_metrics_val['f1_H'].mean()))

In [ ]:
df_metrics_mean = df_metrics_val.mean()

df_metrics_mean = df_metrics_mean.to_frame().T
df_metrics_mean = df_metrics_mean.drop("nfold",axis = 1)

df_metrics_mean

In [ ]:
print("検証データ予測結果")
df_valid_pred.head(10)

In [ ]:
#flatにした辞書を格納するリスト
df_flat_list_tr = []
df_flat_list_val = []
df_flat_list_fin = []

#reportを整理して、各scoreの3foldでの平均値を表示する関数
def report_to_df(report,df_flat_list):
    df_report = pd.DataFrame(report).T
    for col in df_report.columns:  # 0,1,2
        fold = df_report.loc['fold', col]
        report_item = df_report.loc['report', col]
    
        # report_item はすべて辞書なので、直接内部ループ
        for cls, metrics in report_item.items():
            if isinstance(metrics, dict):
                # precision / recall / f1-score がある場合
                df_flat_list.append({
                    "fold": fold,
                    "class": cls,
                    "precision": metrics.get("precision", None),
                    "recall": metrics.get("recall", None),
                    "f1-score": metrics.get("f1-score", None),
                    "accuracy": None
                })
            else:
                # 単一数値 (accuracy)
                df_flat_list.append({
                    "fold": fold,
                    "class": cls,
                    "precision": None,
                    "recall": None,
                    "f1-score": None,
                    "accuracy": metrics
                })
    
    df_flat = pd.DataFrame(df_flat_list)
    df_flat_mean = df_flat.groupby("class")[["precision","recall","f1-score","accuracy"]].mean()

    return df_flat_mean




In [ ]:
print("学習データレポート")
df_flat_mean_tr = report_to_df(report_tr,df_flat_list_tr)
display(df_flat_mean_tr)
print()
print("検証データレポート")
df_flat_mean_val = report_to_df(report_val,df_flat_list_tr)
display(df_flat_mean_val)

# df_metrics_meanを保存

In [ ]:
import json  

# 定数
LOG_FILE_PATH = 'evaluation_results_log.csv'
# 複合的な一意なIDを格納するカラム名
COMPOSITE_ID_COLUMN = 'Composite_ID'

def create_feature_id(feature_list):
    """
    特徴量リストをソートし、カンマ区切りの一意の文字列IDに変換する。
    """
    sorted_features = sorted(feature_list)
    return ",".join(sorted_features)

def create_composite_id(feature_list: list, params: dict) -> str:
    """
    特徴量リストIDとハイパーパラメータを組み合わせて一意の複合IDを作成する。
    """
    # 1. 特徴量IDを作成
    feature_id = create_feature_id(feature_list)
    
    # 2. パラメータをソートし、JSON文字列に変換
    sorted_params = dict(sorted(params.items()))
    params_str = json.dumps(sorted_params, sort_keys=True)
    
    # 3. 複合IDを作成
    # 区切り文字に '|' を使用して結合
    composite_id = f"{feature_id}|{params_str}"
    return composite_id


def update_evaluation_log(feature_list: list, metrics_df: pd.DataFrame, params: dict):
    """
    モデルの評価結果とパラメータをログファイルに追加する。
    特徴量とパラメータの組み合わせが既に存在する場合は、スキップする。
    
    Args:
        feature_list (list): モデルに使用した特徴量のリスト。
        metrics_df (pd.DataFrame): 1行7列の評価指標データフレーム。
        params (dict): モデルに使用したハイパーパラメータの辞書。
    """
    # 1. 複合IDの作成
    composite_id = create_composite_id(feature_list, params)

    # 2. パラメータDataFrameの作成 (ログに含めるため)
    # 辞書を1行のデータフレームに変換
    params_df = pd.DataFrame([params])

    # 3. メタデータ（複合ID、特徴量、タイムスタンプ）データフレームの作成
    metadata_df = pd.DataFrame({
        COMPOSITE_ID_COLUMN: [composite_id], # 重複チェックに使用するID
        'Features': [str(feature_list)],     # 特徴量リスト (視認性のため)
        'Params_JSON': [json.dumps(params)], # パラメータ全体をJSON文字列として保存
        'Timestamp': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')] 
    })
    
    # 4. すべてのデータを結合 (メタデータ + パラメータ詳細 + 評価指標)
    new_result_df = pd.concat([metadata_df, params_df, metrics_df], axis=1)
    
    # 5. ログファイルが存在するかチェック
    if not os.path.exists(LOG_FILE_PATH):
        print("新しいログファイルを作成します。")
        new_result_df.to_csv(LOG_FILE_PATH, index=False)
        return

    # 6. ログファイルが存在する場合: 読み込みと重複チェック
    existing_log_df = pd.read_csv(LOG_FILE_PATH)
    
    # 複合IDが既存のログに含まれているかを確認
    is_duplicate = composite_id in existing_log_df[COMPOSITE_ID_COLUMN].values
    
    if is_duplicate:
        # 重複がある場合はスキップ
        print(f"✅ スキップ: 特徴量とパラメータの組み合わせ '{composite_id}' は既に存在します。")
    else:
        # 重複がない場合は追記
        new_result_df.to_csv(LOG_FILE_PATH, mode='a', header=False, index=False)
        print(f"➡️ 追加: 新しい評価結果をログに追加しました。 ({composite_id})")

In [ ]:
update_evaluation_log(features,df_metrics_mean,params)

# 最終評価（テストデータ）

In [ ]:
x_test = test_df[features]
y_test = test_df[target]

print(x_test.shape)
print(y_test.shape)

In [ ]:
def predict_lgb(input_x,
              input_y,
              original_df,
              list_nfold=[0,1,2]
              ):

    #推論値を格納する変数の作成
    df_test_pred = pd.DataFrame()
    #評価値を入れる変数の作成
    metrics_fin = [] #テストデータ用
    #レポートを入れる変数の作成
    report_fin_list = []
    

    for i,nfold in enumerate(list_nfold):        
        filename = "model_lgb_fold{}.pickle".format(nfold) #学習したモデルの読み込み
        with open(filename,"rb") as f:
          model = pickle.load(f)



        
#----------------------以下、テストデータの評価のコード---------------------------------

        acc_fin,ll_fin,f1_macro_fin,f1_weighted_fin,report_fin,y_pred_fin,y_pred_proba_fin = evaluate_model(model,input_x,input_y)    

        print(f"Fold {i+1} ACC: {acc_fin:.4f}, LogLoss: {ll_fin:.4f}, F1(macro): {f1_macro_fin:.4f}, F1(weighted): {f1_weighted_fin:.4f}")

        #検証スコアを各foldごとに格納 #あとでデータフレーム型に変更
        metrics_fin.append({
            "nfold": nfold,
            "accuracy": acc_fin,
            "log_loss": ll_fin,
            "f1_macro": f1_macro_fin,
            "f1_weighted": f1_weighted_fin,
            # per-class f1 を追加
            "f1_A": report_fin["A"]["f1-score"] if "A" in report_fin else None,
            "f1_D": report_fin["D"]["f1-score"] if "D" in report_fin else None,
            "f1_H": report_fin["H"]["f1-score"] if "H" in report_fin else None,
        })

        report_fin_list.append({
            "fold": nfold,
            "report": report_fin
            })



#----------------検証データの予測値の格納と特徴量重要度の格納---------------------



        #各foldごとに実際の値と予測値を格納したDataFrameを作成

        n = len(input_y)
        
        df_pred_fin = pd.DataFrame({
            "nfold": [nfold] * len(input_y),
            "match_id": list(range(n)),
            "season": original_df["season"],
            "home_team": original_df["home_team"],
            "away_team": original_df["away_team"],
            "true": input_y,
            "pred": y_pred_fin,
            "prob_A": y_pred_proba_fin[:, 0],
            "prob_D": y_pred_proba_fin[:, 1],
            "prob_H": y_pred_proba_fin[:, 2]
        })


        df_test_pred = pd.concat([df_test_pred,df_pred_fin],axis = 0,ignore_index=True)

        
        

#------------------------最終処理---------------------------
    
    print("-"*10,"result","-"*10)
    
    #各foldごとの評価値をデータフレームに格納
    df_metrics_fin = pd.DataFrame(metrics_fin,columns = ["nfold","accuracy","log_loss","f1_macro","f1_weighted","f1_A","f1_D","f1_H"]) #検証データ評価値

    # print("各foldのaccuracyの平均:{:.2f}".format(df_metrics_fin['accuracy'].mean()))
    # print("各foldのlog_lossの平均:{:.2f}".format(df_metrics_fin['log_loss'].mean()))
    # print("各foldのf1_macroの平均:{:.2f}".format(df_metrics_fin['f1_macro'].mean()))
    # print("各foldのf1_weightedの平均:{:.2f}".format(df_metrics_fin['f1_weighted'].mean()))
    # print("各foldのf1_Hの平均:{:.2f}".format(df_metrics_fin['f1_A'].mean()))
    # print("各foldのf1_Aの平均:{:.2f}".format(df_metrics_fin['f1_D'].mean()))
    # print("各foldのf1_Dの平均:{:.2f}".format(df_metrics_fin['f1_H'].mean()))


    return df_test_pred,df_metrics_fin,report_fin_list

In [ ]:
df_test_pred,df_metrics_fin,report_fin = predict_lgb(x_test,y_test,test_df,list_nfold = [0,1,2])

In [ ]:
print("テストデータスコア表")
display(df_metrics_fin)

print("各foldのaccuracyの平均:{:.2f}".format(df_metrics_fin['accuracy'].mean()))
print("各foldのlog_lossの平均:{:.2f}".format(df_metrics_fin['log_loss'].mean()))
print("各foldのf1_macroの平均:{:.2f}".format(df_metrics_fin['f1_macro'].mean()))
print("各foldのf1_weightedの平均:{:.2f}".format(df_metrics_fin['f1_weighted'].mean()))
print("各foldのf1_Hの平均:{:.2f}".format(df_metrics_fin['f1_A'].mean()))
print("各foldのf1_Aの平均:{:.2f}".format(df_metrics_fin['f1_D'].mean()))
print("各foldのf1_Dの平均:{:.2f}".format(df_metrics_fin['f1_H'].mean()))

In [ ]:
print("テストデータ予測結果")
df_test_pred

In [ ]:
df_test_pred_mean = df_test_pred.groupby("match_id")[["prob_A", "prob_D", "prob_H"]].mean()
df_test_pred_mean

In [ ]:
# 3foldの予測確率の平均値から、最終的な勝敗予測のカラムを作る関数
def get_predict(row):
    if (row["prob_A"] > row["prob_D"]) & (row["prob_A"] > row["prob_H"]):
        return "A"
    elif (row["prob_D"] > row["prob_A"]) & (row["prob_D"] > row["prob_H"]):
        return "D"
    elif (row["prob_H"] > row["prob_A"]) & (row["prob_H"] > row["prob_D"]):
        return "H"
    elif (row["prob_H"] == row["prob_A"]) & (row["prob_H"] > row["prob_D"]):
        return "H"
    elif (row["prob_H"] == row["prob_D"]) & (row["prob_H"] > row["prob_A"]):
        return "H"
    elif (row["prob_D"] == row["prob_A"]) & (row["prob_A"] > row["prob_H"]):
        return "A"
    elif (row["prob_H"] == row["prob_A"]) & (row["prob_H"] == row["prob_D"]):
        return "H"

In [ ]:
df_test_pred_mean["pred"] = df_test_pred_mean.apply(get_predict,axis = 1)

In [ ]:
df_test_pred_unique = df_test_pred[["match_id","season","home_team","away_team","true"]].drop_duplicates(subset="match_id")

pred_mean = df_test_pred_unique.merge(df_test_pred_mean, on="match_id", how="inner")

pred_mean

In [ ]:
print("テストデータレポート")
df_flat_mean_fin = report_to_df(report_fin,df_flat_list_fin)
display(df_flat_mean_fin)

#自動化用予測コード

In [ ]:
# def model_study_predict(input_x,input_y,predict_data,params=params):



#予測値を格納する変数の作成
df_valid_pred = pd.DataFrame()


filename = "model_lgb_fold1.pickle" #学習したモデルの読み込み
with open(filename,"rb") as f:
    model = pickle.load(f)

# LightGBM モデル
model = lgb.LGBMClassifier(**params)


# モデルの訓練
model.fit(x_all, y_all)

#保存するモデルのファイル名
filename = "latest_model.pickle"

#モデルを保存
with open(filename, 'wb') as f:
    pickle.dump(model, f, protocol=4)

print("最新モデルの保存が完了しました。")

# ----------------
# 3. 予測の実行 
# ----------------

#予測データの準備
x_predict = x_test



# 予測
y_pred_proba = model.predict_proba(x_predict)
pred_idx = np.argmax(y_pred_proba, axis=1)
y_pred = model.classes_[pred_idx]    


# 4. 結果の出力と保存
x_predict['proba_A'] = y_pred_proba[:,0] 
x_predict['proba_D'] = y_pred_proba[:,1] 
x_predict['proba_H'] = y_pred_proba[:,2] 

x_predict['predicted_result'] = y_pred


In [ ]:
model.classes_

In [ ]:
x_predict[['home_team', 'away_team','proba_A','proba_D','proba_H', 'predicted_result']]

In [ ]:
x_pred = pd.concat([x_predict[['home_team', 'away_team','proba_A','proba_D','proba_H', 'predicted_result']],y_test],axis=1)
x_pred